# Descargas

In [ ]:
!git clone https://github.com/Tio-Panda/INF471-Feria.git

In [ ]:
!pip install --no-input -r ./INF471-Feria/requirements1.txt

# Init

In [ ]:
import sys

EXTENSIONS = [".jpg", ".jpeg", ".png"]
base_path = "/content/INF471-Feria"
sys.path.append(f"{base_path}/code")

In [ ]:
import numpy as np
from numpy.random import default_rng
from img_handlers import plot_imgs_tuple_array, get_imgs_PIL_array, DensityPalette, PrefabricLoader, get_image_from_solution

# Cargar imágenes

In [ ]:
b_single_imgs_path = f"{base_path}/imgs/b_imgs/textures"
b_full_imgs_path = f"{base_path}/imgs/b_imgs/full_textures"

density_palette = DensityPalette(b_single_imgs_path, b_full_imgs_path, 5, 5, 7)
density_palette.show_palette()

In [ ]:
a_imgs_path = f"{base_path}/imgs/a_imgs"

a_imgs_array = get_imgs_PIL_array(a_imgs_path)
plot_imgs_tuple_array(list(enumerate(a_imgs_array)), "index: {}", height=3, width=3)

# Seleccionar imágen

In [ ]:
pf = PrefabricLoader(f"{base_path}/prefabrics")

In [ ]:
name = 1
canvas, shape_map = pf.load_prefabric(name)

In [ ]:
selected_range = (1, -1)
shape_map.set_map_sub_polygons(n_max=50, l_high=8000, l_low=500, range=selected_range)
plot_imgs_tuple_array(list(enumerate([
    canvas.get_canvas(), 
    shape_map.get_map_img((0, -1)), 
    shape_map.get_map_img(selected_range),
    shape_map.get_sub_polygons_img(selected_range)
])), "", height=5, width=8)

# Obtener collage

In [ ]:
original_patch = 2
texture_patch = 1
letter_patch = 3

rng = default_rng(11)
weights = np.array([original_patch, texture_patch, letter_patch])
probs = weights / weights.sum()

original_img = canvas.get_canvas()
N, M = original_img.size

canvas.set_bg_letter(path=f"{base_path}/fonts", font_size=4, gamma=2)

all_polygons = shape_map.get_concatenated_sub_polygons()
n_polygons = len(all_polygons)

polygons_patches = []
for poly in all_polygons:
    choice = rng.choice([0, 1, 2], p=probs)

    if choice == 0:
        polygons_patches.append(poly.get_img_with_texture(canvas, density_palette))
    elif choice == 1:
        polygons_patches.append(canvas.get_masked_img(poly.mask))
    else:
        polygons_patches.append(canvas.get_masked_letter_img(poly.mask))

solution_img = get_image_from_solution(polygons_patches, canvas)

In [ ]:
canvas.set_bg_letter(path=f"{base_path}/fonts", font_size=4, gamma=2)
canvas.set_bg_grid(40, 40, density_palette)

In [ ]:
from img_handlers import get_img_with_bg, row_fusion, col_fusion, diagonal_fusion

original_img = canvas.get_canvas()
w, h = original_img.size

out1 = get_img_with_bg(solution_img, canvas.bg_letter, (255, 255, 255, 255))
out1 = canvas.colorize_img(out1, 0.35)

out2 = get_img_with_bg(solution_img, canvas.bg_grid, (255, 255, 255, 255))
out2 = canvas.colorize_img(out2, 0.15)

out3 = row_fusion(out1, out2, 50, 0.6)

out4 = col_fusion(out1, out2, 100, 0.5)
out4 = row_fusion(out4, canvas.get_canvas(), 50, 0.7)

out5 = diagonal_fusion(out1, out2, 100, 30, 0.5)

out6 = diagonal_fusion(out1, original_img, 90, 40, 0.4)
out6 = diagonal_fusion(out6, out2, 90, -40, 0.6)

plot_imgs_tuple_array(list(enumerate([ 
    solution_img,
    canvas.bg_letter,
    canvas.bg_grid,
    out1,
    out2,
    out3,
    out4,
    out5,
    out6
])), "", height=5, width=8)